### 2.5.2 異なる型のデータを組み合わせる


In [19]:
(define (assoc key records)
  (cond ((null? records) #f)
        ((equal? key (caar records)) (car records))
        (else (assoc key (cdr records)))
        )
  )

(define (lookup key-1 key-2 table)
  (let ((subtable (assoc key-1 (cdr table))))
    (if subtable
        (let ((record (assoc key-2 (cdr subtable))))
          (if record (cdr record)
              #f
              )
          )
        #f
        )
    )
  )

(define (insert! key-1 key-2 value table)
  (let ((subtable (assoc key-1 (cdr table))))
    (if subtable
        (let ((record (assoc key-2 (cdr subtable))))
          (if record (set-cdr! record value)
              (set-cdr! subtable (cons (cons key-2 value) (cdr subtable)))
              )
          )
        (set-cdr! table (cons (list key-1 (cons key-2 value)) (cdr table)))
        )
    )
  'ok)

(define (make-table)
  (let ((local-table (list '*table*)))
    (define (lookup key-1 key-2)
      (let ((subtable (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record (assoc key-2 (cdr subtable))))
              (if record (cdr record) #f))
            #f)
        )
      )
    (define (insert! key-1 key-2 value)
      (let ((subtable (assoc key-1 (cdr local-table))))
        (if subtable
            (let ((record (assoc key-2 (cdr subtable))))
              (if record (set-cdr! record value)
                  (set-cdr! subtable (cons (cons key-2 value) (cdr subtable)))
                  )
              )
            (set-cdr! local-table (cons (list key-1 (cons key-2 value)) (cdr local-table )))
            )
        )
      'ok)
    (define (dispatch m)
      (cond ((eq? m 'lookup-proc) lookup)
            ((eq? m 'insert-proc!) insert!)
            (else (error "Unknown operation: TABLE" m)))
      )
    dispatch)
  )

(define operation-table (make-table))
(define get (operation-table 'lookup-proc))
(define put (operation-table 'insert-proc!))
(define get-coercion get)
(define put-coercion put)

; タグつきデータのコンストラクタ
(define (attach-tag type-tag contents)
  (cons type-tag contents))
; タグを返すセレクタ
(define (type-tag datum)
  (if (pair? datum) (car datum)
      (error "Bad tagged datum: TYPE-TAG" datum))
  )
; データを返すセレクタ
(define (contents datum)
  (if (pair? datum) (cdr datum)
      (error "Bad tagged datum: CONTENTS" datum))
  )

;
(define (apply-generic op . args)
  (let ((type-tags (map type-tag args)))
    (let ((proc (get op type-tags)))
      (if proc
          (apply proc (map contents args))
          (error
           "No method for these types: APPLY-GENERIC"
           (list op type-tags ))))))

In [20]:
(put-coercion 'key1 'key2 "test1")
(put-coercion 'key3 'key4 "test2")


ok

In [21]:
(get-coercion 'key1 'key2)

"test1"

In [22]:
(get-coercion 'key3 'key4)

"test2"

ジェネリック算術演算手続き

In [23]:
(define (add x y) (apply-generic 'add x y))
(define (sub x y) (apply-generic 'sub x y))
(define (mul x y) (apply-generic 'mul x y))
(define (div x y) (apply-generic 'div x y))

schemeの数値

In [24]:
(define (install-scheme-number-package)
  (define (tag x) (attach-tag 'scheme-number x))
  (put 'add '(scheme-number scheme-number) (lambda (x y) (tag (+ x y))))
  (put 'sub '(scheme-number scheme-number) (lambda (x y) (tag (- x y))))
  (put 'mul '(scheme-number scheme-number) (lambda (x y) (tag (* x y))))
  (put 'div '(scheme-number scheme-number) (lambda (x y) (tag (/ x y))))
  (put 'make 'scheme-number (lambda (x) (tag x)))
'done)

(define (make-scheme-number n) ((get 'make 'scheme-number) n))

(install-scheme-number-package)

done

In [25]:
(define a (make-scheme-number 2))
(define b (make-scheme-number 4))

(display (add a b))
(display "\n")
(display (sub a b))
(display "\n")
(display (mul a b))
(display "\n")
(display (div a b))
(display "\n")


(scheme-number . 6)
(scheme-number . -2)
(scheme-number . 8)
(scheme-number . 1/2)


In [26]:
(define (gcd a b)
  (if (= b 0) a
      (gcd b (remainder a b)))
  )

(define (install-rational-package)
  ;; 内部⼿続き
  (define (numer x) (car x))
  (define (denom x) (cdr x))
  (define (make-rat n d)
    (let ((g (gcd n d))) (cons (/ n g) (/ d g))))
  (define (add-rat x y) (make-rat (+ (* (numer x) (denom y))
                                     (* (numer y) (denom x)))
                                  (* (denom x) (denom y))))
  (define (sub-rat x y) (make-rat (- (* (numer x) (denom y))
                                     (* (numer y) (denom x)))
                                  (* (denom x) (denom y))))
  (define (mul-rat x y) (make-rat (* (numer x) (numer y))
                                  (* (denom x) (denom y))))
  (define (div-rat x y) (make-rat (* (numer x) (denom y)) (* (denom x) (numer y))))
  ;; システムのほかの部分とのインターフェイス
  (define (tag x) (attach-tag 'rational x))
  (put 'add '(rational rational) (lambda (x y) (tag (add-rat x y))))
  (put 'sub '(rational rational) (lambda (x y) (tag (sub-rat x y))))
  (put 'mul '(rational rational) (lambda (x y) (tag (mul-rat x y)))) 
  (put 'div '(rational rational) (lambda (x y) (tag (div-rat x y))))
  (put 'make 'rational (lambda (n d) (tag (make-rat n d))))
  'done)

(define (make-rational n d) ((get 'make 'rational) n d))

(install-rational-package)

done

In [27]:
(define a (make-rational 1 2))
(define b (make-rational 1 3))

(display (add a b))
(display "\n")
(display (sub a b))
(display "\n")
(display (mul a b))
(display "\n")
(display (div a b))
(display "\n")


(rational 5 . 6)
(rational 1 . 6)
(rational 1 . 6)
(rational 3 . 2)


In [28]:
(import "math")
(define (make-from-mag-ang r a)
  (define (dispatch op)
    (cond ((eq? op 'real-part) (* r (math.cos a)))
          ((eq? op 'imag-part) (* r (math.sin a)))
          ((eq? op 'magnitude) r)
          ((eq? op 'angle) a)
          (else (error "Unknown op: MAKE-FROM-MAG-ANG" op))
          )
  )
dispatch)

(define (square x)(* x x))

(define (install-rectangular-package)
  ;; 内部⼿続き
  (define (real-part z) (car z))
  (define (imag-part z) (cdr z))
  (define (make-from-real-imag x y) (cons x y))
  (define (magnitude z) (sqrt (+ (square (real-part z)) (square (imag-part z)))))
  (define (angle z) (math.atan2 (imag-part z) (real-part z)))
  (define (make-from-mag-ang r a) (cons (* r (math.cos a)) (* r (math.sin a))))
  ;; システムのほかの部分とのインターフェイス
  (define (tag x) (attach-tag 'rectangular x))
  (put 'real-part '(rectangular) real-part)
  (put 'imag-part '(rectangular) imag-part)
  (put 'magnitude '(rectangular) magnitude)
  (put 'angle '(rectangular) angle)
  (put 'make-from-real-imag 'rectangular (lambda (x y) (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'rectangular (lambda (r a) (tag (make-from-mag-ang r a))))
'done)

(define (install-polar-package)
  ;; 内部⼿続き
  (define (magnitude z) (car z))
  (define (angle z) (cdr z))
  (define (make-from-mag-ang r a) (cons r a))
  (define (real-part z) (* (magnitude z) (math.cos (angle z))))
  (define (imag-part z) (* (magnitude z) (math.sin (angle z))))
  (define (make-from-real-imag x y) (cons (sqrt (+ (square x) (square y))) (math.atan2 y x)))
  ;; システムのほかの部分とのインターフェイス
  (define (tag x) (attach-tag 'polar x))
  (put 'real-part '(polar) real-part)
  (put 'imag-part '(polar) imag-part)
  (put 'magnitude '(polar) magnitude)
  (put 'angle '(polar) angle)
  (put 'make-from-real-imag 'polar (lambda (x y) (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'polar (lambda (r a) (tag (make-from-mag-ang r a))))
  'done)

(define (real-part z) (apply-generic 'real-part z))
(define (imag-part z) (apply-generic 'imag-part z))
(define (magnitude z) (apply-generic 'magnitude z))
(define (angle z) (apply-generic 'angle z))

(define (make-from-real-imag x y) ((get 'make-from-real-imag 'rectangular) x y))
(define (make-from-mag-ang r a) ((get 'make-from-mag-ang 'polar) r a))

(install-rectangular-package)
(install-polar-package)

done

In [29]:
(define (install-complex-package)
  ;; 直交形式パッケージと極形式パッケージからインポートした⼿続き
  (define (make-from-real-imag x y) ((get 'make-from-real-imag 'rectangular) x y))
  (define (make-from-mag-ang r a) ((get 'make-from-mag-ang 'polar) r a))
  ;; 内部⼿続き
  (define (add-complex z1 z2) (make-from-real-imag (+ (real-part z1) (real-part z2)) (+ (imag-part z1) (imag-part z2))))
  (define (sub-complex z1 z2) (make-from-real-imag (- (real-part z1) (real-part z2)) (- (imag-part z1) (imag-part z2))))
  (define (mul-complex z1 z2) (make-from-mag-ang (* (magnitude z1) (magnitude z2)) (+ (angle z1) (angle z2))))
  (define (div-complex z1 z2) (make-from-mag-ang (/ (magnitude z1) (magnitude z2)) (- (angle z1) (angle z2))))
  ;; システムのほかの部分とのインターフェイス
  (define (tag z) (attach-tag 'complex z))
  (put 'add '(complex complex) (lambda (z1 z2) (tag (add-complex z1 z2))))
  (put 'sub '(complex complex) (lambda (z1 z2) (tag (sub-complex z1 z2))))
  (put 'mul '(complex complex) (lambda (z1 z2) (tag (mul-complex z1 z2))))
  (put 'div '(complex complex) (lambda (z1 z2) (tag (div-complex z1 z2))))
  (put 'make-from-real-imag 'complex (lambda (x y) (tag (make-from-real-imag x y))))
  (put 'make-from-mag-ang 'complex (lambda (r a) (tag (make-from-mag-ang r a))))
  'done)

(define (make-complex-from-real-imag x y) ((get 'make-from-real-imag 'complex) x y))
(define (make-complex-from-mag-ang r a) ((get 'make-from-mag-ang 'complex) r a))

(install-complex-package)

done

In [30]:
(define a (make-complex-from-real-imag 1 2))
(define b (make-complex-from-real-imag 1 3))

(display (add a b))
(display "\n")
(display (sub a b))
(display "\n")
(display (mul a b))
(display "\n")
(display (div a b))
(display "\n")


(complex rectangular 2 . 5)
(complex rectangular 0 . -1)
(complex polar 7.071067811865476 . 2.356194490192345)
(complex polar 0.7071067811865475 . -0.14189705460416402)


強制型変換

In [31]:
(define (scheme-number->complex n)
  (make-complex-from-real-imag (contents n) 0))
(put-coercion 'scheme-number 'complex scheme-number->complex)

ok

In [32]:
(get-coercion 'scheme-number 'complex)

#<procedure>

In [33]:
(define (apply-generic op . args)
  (let ((type-tags (map type-tag args)))
    (let ((proc (get op type-tags)))
      (if proc
          (apply proc (map contents args))
          (if (= (length args) 2)
              (let ((type1 (car type-tags))
                    (type2 (cadr type-tags))
                    (a1 (car args))
                    (a2 (cadr args)))
                (let ((t1->t2 (get-coercion type1 type2))
                      (t2->t1 (get-coercion type2 type1)))
                  (cond (t1->t2
                         (apply-generic op (t1->t2 a1) a2))
                        (t2->t1
                         (apply-generic op a1 (t2->t1 a2)))
                        (else (error "No method for these types"
                                     (list op type-tags ))))))
              (error "No method for these types"
                     (list op type-tags )))))))

In [34]:
(define z (make-complex-from-real-imag 1 3))
(define n (make-scheme-number 5))

In [35]:
(add z n)

(complex rectangular 6 . 3)

#### 練習問題

- [練習問題2.81](../exercises/2.81.ipynb)
- [練習問題2.82](../exercises/2.82.ipynb)
- [練習問題2.83](../exercises/2.83.ipynb)
- [練習問題2.84](../exercises/2.84.ipynb)
- [練習問題2.85](../exercises/2.85.ipynb)
- [練習問題2.86](../exercises/2.86.ipynb)